[View in Colaboratory](https://colab.research.google.com/github/peterjsadowski/keras_tutorial/blob/master/4_keras_mnist_SHERPA.ipynb)

In [1]:
"""
SHERPA is a Python library for hyperparameter tuning of machine learning models.
Copyright (C) 2018  Lars Hertel, Peter Sadowski, and Julian Collado.

This file is part of SHERPA.

SHERPA is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

SHERPA is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with SHERPA.  If not, see <http://www.gnu.org/licenses/>.

INSTALLATION: 
pip install parameter-sherpa
pip install flask==0.12.2 # Newer version of flash leads to error: 'io.UnsupportedOperation: not writable'
"""
!pip install parameter-sherpa
!pip install flask==0.12.2

from __future__ import print_function
import sherpa
import time
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import mnist

    100% |████████████████████████████████| 532kB 5.7MB/s 
    100% |████████████████████████████████| 409kB 6.0MB/s 
    100% |████████████████████████████████| 92kB 9.8MB/s 
    100% |████████████████████████████████| 51kB 13.9MB/s 
    100% |████████████████████████████████| 81kB 9.5MB/s 
  Running setup.py bdist_wheel for itsdangerous ... - done
  Stored in directory: /root/.cache/pip/wheels/2c/4a/61/5599631c1554768c6290b08c02c72d7317910374ca602ff1e5
Successfully built itsdangerous
    100% |████████████████████████████████| 92kB 2.9MB/s 
  Found existing installation: Flask 1.0.2
    Uninstalling Flask-1.0.2:
      Successfully uninstalled Flask-1.0.2


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
num_classes = 10

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [0]:
parameters = [sherpa.Discrete('num_units', [16, 32])]

In [0]:
algorithm = sherpa.algorithms.BayesianOptimization(max_num_trials=50)

In [0]:
study = sherpa.Study(parameters=parameters,
                     algorithm=algorithm,
                     lower_is_better=True)

INFO:sherpa.core:
-------------------------------------------------------
SHERPA Dashboard running on http://127.0.1.1:8880
-------------------------------------------------------


In [0]:
batch_size = 32
epochs = 10

for trial in study:
    print("Trial {}:\t{}".format(trial.id, trial.parameters))

    model = Sequential()
    model.add(Dense(units=trial.parameters['num_units'], activation='relu', input_dim=784))
    model.add(Dense(units=10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              callbacks=[study.keras_callback(trial, objective_name='val_loss')],
              validation_data=(x_test, y_test))

    study.finalize(trial=trial)

Trial 1:	{'num_units': 64}
Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 4s 75us/step - loss: 0.3021 - acc: 0.9155 - val_loss: 0.1648 - val_acc: 0.9521
Epoch 2/15
60000/60000 [==============================] - 5s 78us/step - loss: 0.1451 - acc: 0.9576 - val_loss: 0.1268 - val_acc: 0.9630
Epoch 3/15
60000/60000 [==============================] - 4s 75us/step - loss: 0.1062 - acc: 0.9697 - val_loss: 0.1138 - val_acc: 0.9649
Epoch 4/15
60000/60000 [==============================] - 4s 74us/step - loss: 0.0845 - acc: 0.9750 - val_loss: 0.0997 - val_acc: 0.9682
Epoch 5/15
60000/60000 [==============================] - 4s 70us/step - loss: 0.0688 - acc: 0.9786 - val_loss: 0.0896 - val_acc: 0.9736
Epoch 6/15
60000/60000 [==============================] - 4s 70us/step - loss: 0.0569 - acc: 0.9825 - val_loss: 0.0865 - val_acc: 0.9736
Epoch 7/15
60000/60000 [==============================] - 4s 70us/step - loss: 0.0486 - acc: 0.9850 -

In [0]:
print(study.get_best_result())

{'Trial-ID': 9, 'Iteration': 5, 'num_units': 125, 'Objective': 0.06545425380312372}
